In [ ]:
from framed.io_utils.sbml import load_cbmodel
from pickle import dump, load
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
%run '../src/straindesign.py'

## Load (customized) iAF1260 model 

In [ ]:
model = load_cbmodel('../../models/iAF1260_MCSEnum.xml', flavor='cobra')

biomass = model.detect_biomass_reaction()
product = 'R_EX_etoh_e'
oxygen = 'R_EX_o2_e'
glucose = 'R_EX_glc_e'
objective = lambda v: v[product]
min_growth = 0.001
max_uptake = 10
min_yield = 1.4
min_prod = min_yield*max_uptake
model.set_lower_bound(oxygen, 0)
model.set_lower_bound(glucose, -max_uptake)
constraints={biomass: (min_growth, 1000), product: (min_prod, 1000)}

## Load MCSEnumerator results (scenario 1)

In [ ]:
data = loadmat('../../results/mcs/mcs_iAF1260_MCSEnum_max7.mat')
cmcs = pd.DataFrame(data['cmcs'].T, index=model.reactions.keys())
rxn_dels = [cmcs[cmcs[col] == 1].index.tolist() for col in cmcs]

## Compute gene based solutions 

In [ ]:
parse_model_rules(model)

In [ ]:
%time rxns2genes = expand_gene_sets(model, rxn_dels)

In [ ]:
%time rxns2rxns = gene_to_reaction_sets(model, rxns2genes)

In [ ]:
%time solution_pool = build_reaction_solution_pool(model, rxns2rxns, constraints)

## Analyse results

In [ ]:
sizes = sorted(set(map(len, rxns2genes.keys())))

columns = ['cMCSs', 'knockable', 'gene-wise total', 'reaction-wise total', 
           'feasible', 'gene deletion size', 'reaction deletion size']

cMCS_count = []
knockable_count = []
gene_set_count = []
true_rxns_count = []
valid_rxns_count = []
invalid_rxns_count = []
gene_set_sizes = []
rxn_set_sizes = []
stats = pd.DataFrame([], columns=columns)

for n_dels in sizes:
    total_sets = [rxn_set for rxn_set in rxn_dels if len(rxn_set) == n_dels]
    cMCS_count.append(len(total_sets))

    knockable_sets = [rxn_set for rxn_set in rxns2genes.keys() if len(rxn_set) == n_dels]
    knockable_count.append(len(knockable_sets))

    gene_sets = set([gene_set for rxn_set in knockable_sets 
                     for gene_set in rxns2genes[rxn_set]])
    gene_set_count.append(len(gene_sets))

    true_rxn_sets = set([true_rxn_set for rxn_set in knockable_sets 
                         for true_rxn_set in rxns2rxns[rxn_set]])
    true_rxns_count.append(len(true_rxn_sets))

    valid_rxn_sets = [rxn_set for rxn_set in true_rxn_sets 
                      if solution_pool[rxn_set].valid]
    valid_rxns_count.append(len(valid_rxn_sets))

    invalid_rxn_sets = [rxn_set for rxn_set in true_rxn_sets 
                        if not solution_pool[rxn_set].valid]
    invalid_rxns_count.append(len(invalid_rxn_sets))

    gene_set_size = map(len, gene_sets)
    gene_set_sizes.append(gene_set_size)

    rxn_set_size = map(len, true_rxn_sets)
    rxn_set_sizes.append(rxn_set_size)

    stats.loc[n_dels] = (len(total_sets),
                      len(knockable_sets),
                      len(gene_sets),
                      len(true_rxn_sets),
                      len(valid_rxn_sets),
                      '[{} - {}]'.format(min(gene_set_size), max(gene_set_size)),
                      '[{} - {}]'.format(min(rxn_set_size), max(rxn_set_size)))

stats.index.name = 'n dels'

## Store statistics as table

In [ ]:
stats.to_csv('../results/mcs/mcs_iAF1260_MCSEnum_summary.csv')

## Generate plots

In [ ]:
colors = sns.color_palette()

plt.figure(figsize=(4,3))
sns.barplot(sizes, cMCS_count, log=True, color=colors[0])
plt.xlabel('cut set size')
plt.ylabel('# of cMCS')
plt.tight_layout()
plt.savefig('../results/mcs/plots/cmcs_size.png', dpi=150)

In [ ]:
plt.figure(figsize=(4,3))
sns.barplot(sizes, gene_set_count, log=True, color=colors[0])
plt.xlabel('cut set size')
plt.ylabel('# of gene-wise solutions')
plt.yticks([1e0, 1e2, 1e4, 1e6, 1e8])
plt.tight_layout()
plt.savefig('../results/mcs/plots/gene_sol_size.png', dpi=150)

In [ ]:
plt.figure(figsize=(4,3))
df = pd.DataFrame({'feasible': valid_rxns_count, 'infeasible': invalid_rxns_count, 'size': sizes})
df = pd.melt(df, id_vars=['size'])
df.columns = ['size', '', 'count']
sns.barplot(data=df, x='size', y='count', hue='', log=True, palette=colors[1:3])
plt.xlabel('cut set size')
plt.ylabel('# of solutions')
plt.tight_layout()
plt.savefig('../results/mcs/plots/rxn_sol_size.png', dpi=150)

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.boxplot(pd.DataFrame(gene_set_sizes, index=sizes).T, fliersize=5, sym='o', color=colors[0])
plt.xlabel('cut set size')
plt.ylabel('gene deletion size')
plt.tight_layout()
plt.ylim((0, 22))
plt.savefig('../results/mcs/plots/gene_cut_size.png', dpi=150)

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.boxplot(pd.DataFrame(rxn_set_sizes, index=sizes).T, fliersize=5, sym='o', color=colors[0])
plt.xlabel('cut set size')
plt.ylabel('reaction deletion size')
plt.yticks([0, 100, 200, 300])
plt.tight_layout()
plt.savefig('../results/mcs/plots/rxn_cut_size.png', dpi=150)